# Mistplay Fraud Demo – Batch Inference

This notebook scores recent accounts using the registered model and writes risk scores to a Delta table for analyst review.

In [0]:
!pip install -q databricks-feature-store
!pip install scikit-learn==1.7.2
dbutils.library.restartPython()

In [0]:
from databricks.feature_store import FeatureStoreClient
from pyspark.sql import functions as F

DB_NAME = "ramin_serverless_aws_catalog.mistplay_fraud_demo"

fs = FeatureStoreClient()

account_device = spark.table(f"{DB_NAME}.account_device")
events = spark.table(f"{DB_NAME}.events")

# best_model = (
#     spark.table(f"{DB_NAME}.model_selection")
#     .orderBy(F.col("selected_at").desc())
#     .select("model_name")
#     .first()
# )

# if not best_model:
#     raise ValueError("No model selection found. Run the training notebook first.")

# model_name = best_model["model_name"]
model_name = 'ramin_serverless_aws_catalog.mistplay_fraud_demo.mistplay_fraud_model_lr'


# Simulate a 1-day lookback of active accounts
recent_accounts = (
    events.filter(F.col("event_ts") >= F.expr("current_timestamp() - interval 10 day"))
    .select("account_id")
    .distinct()
)

scoring_input = recent_accounts.join(account_device, on="account_id", how="left")

model_uri = f"models:/{model_name}/3"

predictions = fs.score_batch(model_uri=model_uri, df=scoring_input)

scored = (
    predictions
    .withColumnRenamed("prediction", "fraud_score")
    .withColumn(
        "risk_bucket",
        F.when(F.col("fraud_score") >= 0.8, F.lit("high"))
        .when(F.col("fraud_score") >= 0.5, F.lit("medium"))
        .otherwise(F.lit("low")),
    )
    .withColumn("scored_at", F.current_timestamp())
)

scored.write.mode("overwrite").saveAsTable(f"{DB_NAME}.fraud_scores_batch")

print("Batch scores saved to:")
print(f"- {DB_NAME}.fraud_scores_batch")
print("Model used:")
print(f"- {model_name}")